<!-- Level 1 and Level 2 -->

### Level 1 and Level 2

In [ ]:
!pip install timm scipy scikit-learn --quiet


In [ ]:
import os
import numpy as np
import pandas as pd
import torch
import timm
import scipy.io
import matplotlib.pyplot as plt

from PIL import Image
from sklearn.model_selection import StratifiedShuffleSplit
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Device:", device)


Device: cuda


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

import zipfile

ZIP_PATH = "/content/drive/MyDrive/cars196/dataset/archive (7).zip"
BASE_DIR = "/content/cars196"

os.makedirs(BASE_DIR, exist_ok=True)

with zipfile.ZipFile(ZIP_PATH, 'r') as zip_ref:
    zip_ref.extractall(BASE_DIR)

print("Extracted:", os.listdir(BASE_DIR))


Mounted at /content/drive
Extracted: ['car_devkit', 'cars_test', 'cars_train']


In [ ]:
TRAIN_IMG_DIR = "/content/cars196/cars_train/cars_train"
TEST_IMG_DIR  = "/content/cars196/cars_test/cars_test"

DEVKIT = "/content/cars196/car_devkit/devkit"
TRAIN_ANNO = DEVKIT + "/cars_train_annos.mat"
TEST_ANNO  = DEVKIT + "/cars_test_annos.mat"



In [ ]:
import scipy.io

mat = scipy.io.loadmat(TEST_ANNO)

print(mat.keys())
print(type(mat["annotations"]))
print(len(mat["annotations"][0]))

# print first annotation fully
print(mat["annotations"][0][0])


dict_keys(['__header__', '__version__', '__globals__', 'annotations'])
<class 'numpy.ndarray'>
8041
(array([[30]], dtype=uint8), array([[52]], dtype=uint8), array([[246]], dtype=uint8), array([[147]], dtype=uint8), array(['00001.jpg'], dtype='<U9'))


In [ ]:
def load_annotations(mat_file, img_dir, has_labels=True):
    mat = scipy.io.loadmat(mat_file)
    annotations = mat['annotations'][0]

    data = []
    for ann in annotations:
        x1 = int(ann[0][0][0])
        y1 = int(ann[1][0][0])
        x2 = int(ann[2][0][0])
        y2 = int(ann[3][0][0])

        if has_labels:
            label = int(ann[4][0][0]) - 1
            fname = ann[5][0]
        else:
            label = -1   # dummy
            fname = ann[4][0]

        data.append({
            "image": os.path.join(img_dir, fname),
            "label": label,
            "bbox": (x1, y1, x2, y2)
        })

    return pd.DataFrame(data)


In [ ]:
train_df_full = load_annotations(TRAIN_ANNO, TRAIN_IMG_DIR, has_labels=True)
test_df = load_annotations(TEST_ANNO, TEST_IMG_DIR, has_labels=False)

print("Train:", len(train_df_full))
print("Test:", len(test_df))


Train: 8144
Test: 8041


In [ ]:
splitter = StratifiedShuffleSplit(n_splits=1, test_size=0.2, random_state=42)

train_idx, val_idx = next(splitter.split(train_df_full["image"], train_df_full["label"]))

train_df = train_df_full.iloc[train_idx].reset_index(drop=True)
val_df   = train_df_full.iloc[val_idx].reset_index(drop=True)

print("Train:", len(train_df))
print("Val:", len(val_df))
print("Test:", len(test_df))


Train: 6515
Val: 1629
Test: 8041


In [ ]:
class CarsDataset(Dataset):
    def __init__(self, df, transform=None):
        self.df = df
        self.transform = transform

    def __len__(self):
        return len(self.df)

    def __getitem__(self, idx):
        row = self.df.iloc[idx]
        img = Image.open(row["image"]).convert("RGB")

        x1, y1, x2, y2 = row["bbox"]
        img = img.crop((x1, y1, x2, y2))

        label = row["label"]

        if self.transform:
            img = self.transform(img)

        return img, label


In [ ]:
train_tf = transforms.Compose([
    transforms.Resize((380, 380)),
    transforms.RandomHorizontalFlip(),
    transforms.ColorJitter(0.2,0.2,0.2,0.1),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],
                         [0.229,0.224,0.225])
])

val_tf = transforms.Compose([
    transforms.Resize((380, 380)),
    transforms.ToTensor(),
    transforms.Normalize([0.485,0.456,0.406],
                         [0.229,0.224,0.225])
])


In [ ]:
train_ds = CarsDataset(train_df, train_tf)
val_ds   = CarsDataset(val_df, val_tf)
test_ds  = CarsDataset(test_df, val_tf)

train_loader = DataLoader(train_ds, batch_size=16, shuffle=True, num_workers=2)
val_loader   = DataLoader(val_ds, batch_size=16, shuffle=False, num_workers=2)
test_loader  = DataLoader(test_ds, batch_size=16, shuffle=False, num_workers=2)


In [ ]:
model = timm.create_model("efficientnet_b4", pretrained=True, num_classes=196)
model = model.to(device)

criterion = torch.nn.CrossEntropyLoss(label_smoothing=0.1)
optimizer = torch.optim.AdamW(model.parameters(), lr=3e-4, weight_decay=1e-4)
scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(optimizer, T_max=30)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/77.9M [00:00<?, ?B/s]

In [ ]:
def train_epoch(model, loader):
    model.train()
    total, correct, loss_sum = 0, 0, 0

    for x, y in loader:
        x, y = x.to(device), y.to(device)

        optimizer.zero_grad()
        preds = model(x)
        loss = criterion(preds, y)

        loss.backward()
        optimizer.step()

        loss_sum += loss.item()
        correct += (preds.argmax(1) == y).sum().item()
        total += y.size(0)

    return loss_sum/len(loader), correct/total


def eval_epoch(model, loader):
    model.eval()
    total, correct, loss_sum = 0, 0, 0

    with torch.no_grad():
        for x, y in loader:
            x, y = x.to(device), y.to(device)

            preds = model(x)
            loss = criterion(preds, y)

            loss_sum += loss.item()
            correct += (preds.argmax(1) == y).sum().item()
            total += y.size(0)

    return loss_sum/len(loader), correct/total


In [ ]:
EPOCHS = 30
train_accs, val_accs = [], []

for epoch in range(EPOCHS):
    train_loss, train_acc = train_epoch(model, train_loader)
    val_loss, val_acc = eval_epoch(model, val_loader)

    scheduler.step()

    train_accs.append(train_acc)
    val_accs.append(val_acc)

    print(f"Epoch {epoch+1:02d} | Train Acc: {train_acc:.4f} | Val Acc: {val_acc:.4f}")


Epoch 01 | Train Acc: 0.2513 | Val Acc: 0.6771
Epoch 02 | Train Acc: 0.8493 | Val Acc: 0.8392
Epoch 03 | Train Acc: 0.9679 | Val Acc: 0.8521
Epoch 04 | Train Acc: 0.9883 | Val Acc: 0.8711
Epoch 05 | Train Acc: 0.9940 | Val Acc: 0.8870
Epoch 06 | Train Acc: 0.9955 | Val Acc: 0.8956
Epoch 07 | Train Acc: 0.9962 | Val Acc: 0.9024
Epoch 08 | Train Acc: 0.9963 | Val Acc: 0.9079
Epoch 09 | Train Acc: 0.9974 | Val Acc: 0.9012
Epoch 10 | Train Acc: 0.9966 | Val Acc: 0.9061
Epoch 11 | Train Acc: 0.9972 | Val Acc: 0.9122
Epoch 12 | Train Acc: 0.9977 | Val Acc: 0.9091
Epoch 13 | Train Acc: 0.9979 | Val Acc: 0.9147
Epoch 14 | Train Acc: 0.9975 | Val Acc: 0.9091
Epoch 15 | Train Acc: 0.9985 | Val Acc: 0.9147
Epoch 16 | Train Acc: 0.9977 | Val Acc: 0.9165
Epoch 17 | Train Acc: 0.9983 | Val Acc: 0.9202
Epoch 18 | Train Acc: 0.9980 | Val Acc: 0.9128
Epoch 19 | Train Acc: 0.9988 | Val Acc: 0.9165
Epoch 20 | Train Acc: 0.9982 | Val Acc: 0.9190
Epoch 21 | Train Acc: 0.9985 | Val Acc: 0.9190
Epoch 22 | Tr

In [ ]:
import torch

SAVE_PATH = "/content/drive/MyDrive/terafac_models/efficientnet_cars196_final.pth"
os.makedirs(os.path.dirname(SAVE_PATH), exist_ok=True)

torch.save(model.state_dict(), SAVE_PATH)
print("✅ Model saved to:", SAVE_PATH)


✅ Model saved to: /content/drive/MyDrive/terafac_models/efficientnet_cars196_final.pth


In [ ]:
import json

history = {
    "train_acc": train_accs,
    "val_acc": val_accs
}

HISTORY_PATH = "/content/drive/MyDrive/terafac_models/history.json"
with open(HISTORY_PATH, "w") as f:
    json.dump(history, f)

print("✅ Training history saved to:", HISTORY_PATH)


✅ Training history saved to: /content/drive/MyDrive/terafac_models/history.json


In [ ]:
RESUME_PATH = "/content/drive/MyDrive/terafac_models/checkpoint.pth"

torch.save({
    "model_state_dict": model.state_dict(),
    "optimizer_state_dict": optimizer.state_dict(),
    "scheduler_state_dict": scheduler.state_dict(),
    "train_accs": train_accs,
    "val_accs": val_accs
}, RESUME_PATH)

print("✅ Full checkpoint saved to:", RESUME_PATH)


✅ Full checkpoint saved to: /content/drive/MyDrive/terafac_models/checkpoint.pth
